In [ ]:
# Project title: Meal Suggestion App
# Name: Sheldon Gordon


import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import random
import re


def main():
    recipe_df = access_csv('recipe_data.csv')
    recipe_dict = create_recipe_dict(recipe_df)
    meal_choice(recipe_dict)


def access_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError:
        print(f'Error: File at "{file_path}" not found.')
        return pd.DataFrame()
    except Exception as e:
        print(f'Error reading CSV: {e}')
        return pd.DataFrame()


def create_recipe_dict(d):
    recipe_dict = {row["recipe_name"].lower(): row.to_dict() for _, row in d.iterrows()}
    return recipe_dict


def meal_choice(recipe_dict):
    while True:
            meal = input('What would you like to eat (or type "exit" to quit)?: ').strip().lower()

            if meal == 'exit':
                print('Goodbye!')
                return 'Program exited!'

            recipe = recipe_dict.get(meal)
            if recipe:
                print(f'Great choice. Here is the recipe for {meal.title()}:')
                print(f"Ingredients:\n{recipe['ingredients']}\n")
                print(f"Directions:\n{recipe['directions']}\n")
                prompt_for_email(meal, recipe)
                return {'status': 'success'}
            else:
                print('Unfortunately your meal choice is not available currently.')
                recipe
                suggest_alternatives(recipe_dict)
                break


def suggest_alternatives(recipe_dict):
    while True:
        random_suggestions = random.sample(list(recipe_dict.keys()), min(3, len(list(recipe_dict.keys()))))
        print('Here are some suggestions:')
        for i, suggestion in enumerate(random_suggestions, 1):
            print(f'{i}. {suggestion.title()}')
        print('4. Enter your own choice.')

        choice = input('Select a number from the options above: ').strip()

        if choice.isdigit() and 1 <= int(choice) <= 3:
            selected_meal = random_suggestions[int(choice) - 1]
            recipe = recipe_dict[selected_meal]
            print(f'Great choice. Here is the recipe for {selected_meal.title()}:')
            print(f"Ingredients:\n{recipe['ingredients']}\n")
            print(f"Directions:\n{recipe['directions']}\n")
            prompt_for_email(selected_meal, recipe)
            return {'status': 'success'}
        elif choice == '4':
            own_choice = meal_choice(recipe_dict)
            return
        else:
            print('Invalid choice. Please try again.')
            continue


def prompt_for_email(recipe_name, recipe):
    to_email = input('Would you also like the recipe sent to your email? (yes/no): ').strip().lower()
    if to_email == 'yes':
        email = input('Please enter your email address: ').strip().lower()
        if validate_email(email):
            send_email(email, recipe_name, recipe)
            return True
        else:
            print('Invalid email format. Please try again.')
    return False


def validate_email(email):
    pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    match = re.match(pattern, email)
    if match:
        return 'Valid'
    else:
        return 'Invalid'


def send_email(email, recipe_name, recipe):
    try:
        sender_email = 'sxxxxgxxxx4@gmail.com'
        sender_password = 'xxxx xxxx xxxx'
        subject = f'Recipe for {recipe_name}'

        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = email
        message['Subject'] = subject

        body = f"""
        Recipe Name: {recipe['recipe_name']}\n
        Ingredients:\n{recipe['ingredients']}\n
        Directions:\n{recipe['directions']}\n
        """

        message.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.send_message(message)
        print(f'The recipe has been sent to {email}!')
    except Exception as e:
        print(f'Error sending email: {e}')


if __name__ == "__main__":
    main()
